**Author:** Elias Rafael Ruiz-Morales

**Institution:** Wellcome Sanger institute

**July, 2023**

---

# Creation of tables to generate differential gene expression heatmaps - Trophoblasts

In [8]:
recoverPvalLog2FC <- function(files, columns, genes, outdir, suffix){
    # Function to get the Pvalues and log2FC values from the Differential expression results.
    # Useful to make plots for downstream analysis.
    
    # files= files from the differential expression results
    # columns= column names, each column name MUST MATCH to a file in the same position
    # genes= query genes to recover their values
    # outdir= output directory
    # suffix= suffix to add to the file names.
    #-------------------------------------------
    
    
    ## Building the OUTPUT dataframes
    ##

    #dimensions= columns * genes
    pval= matrix(data = rep(x = NA,(length(columns)*length(genes))),ncol = length(columns) )
    rownames(pval)=genes
    colnames(pval)=columns

    #Same structure for the log2FC datafram
    log2FC= pval
    
    ## Filling up the empty data frames
    ##
    for(i in 1:length(columns)){

        #column and file names that MUST be in the same position in both vectors

        file= files[i] #DE file to read
        column=columns[i] #column to fill

        #reading DE dataframe
        df= read.csv(file, sep = '\t')
        rownames(df)=df[,1] #rownames (genes) and deleting extra column with the rownames
        df[,1]=NULL


        ## Filling up the column, gene by gene
        ##
        for(gene in genes){  
            #checking if a gene is in the DE dataframe 
            if (gene%in%rownames(df)){

                    pval[gene,column]= df[gene,'p_val_adj']
                    log2FC[gene,column]= df[gene,'logFC']   

   
            }
            else{
                #gene is not in DE dataframe, place a pvalue=1 and log2FC=0
                pval[gene,column]= 1
                log2FC[gene,column]=0
            }
        }
    }

    if(missing(suffix)){
        file_name= paste0(outdir,'padj_selectedGenes.csv', collapse = '')
        write.csv(pval, file=file_name, quote=FALSE, row.names=TRUE)
        
        file_name= paste0(outdir,'log2FC_selectedGenes.csv', collapse = '')
        write.csv(log2FC, file=file_name, quote=FALSE, row.names=TRUE)
    }
    else{
        file_name= paste0(outdir,'padj_selectedGenes',suffix, collapse = '')
        write.csv(pval,file=file_name, quote=FALSE, row.names=TRUE)
        
        file_name= paste0(outdir,'log2FC_selectedGenes',suffix, collapse = '')
        write.csv(log2FC,file=file_name, quote=FALSE, row.names=TRUE)
    }
  
    
}


In [9]:
#files containing the tables with the DEGs from each cell type per infection
files= c('../results/SC_limma_Trophoblast_MAL_24h/1_DEGs_SC_MAL_24h_VCT_fusing.tsv',
         '../results/SC_limma_Trophoblast_MAL_24h/2_DEGs_SC_MAL_24h_VCT.tsv',
         
         '../results/SC_limma_Trophoblast_LIS_24h/1_DEGs_SC_LIS_24h_VCT_fusing.tsv',
         '../results/SC_limma_Trophoblast_LIS_24h/2_DEGs_SC_LIS_24h_VCT.tsv',
                  
         '../results/SC_limma_Trophoblast_TOX_24h/1_DEGs_SC_TOX_24h_VCT_fusing.tsv',
         '../results/SC_limma_Trophoblast_TOX_24h/2_DEGs_SC_TOX_24h_VCT.tsv'
        )


#cell types per infections that will be plot. This MUST match with the files above
columns=c("PF_VCT_f","PF_VTC",#"PF_HBinf",
          "LM_VCT_f","LM_VTC",#"LM_HBinf",
          "TG_VCT_f","TG_VTC")#,"TG_HBinf",)


-----

## Inflammation genes shared among compartments, across infections - Figure 2

In [10]:
genes_input= c(
    #--- Inflammation
    "IL1B","CCL3","CCL4","CCL20","CXCL8","CXCL3","PTGS2"

)
recoverPvalLog2FC(files, columns, genes=genes_input, outdir='../results/tables_limma/', suffix = '_selectedSharedInflammation.csv')

## HMOX1 gene, specific to P. falciparum infection - Figure 3

In [4]:
genes_input= c(
    "HMOX1"
)
recoverPvalLog2FC(files, columns, genes=genes_input, outdir='../results/tables_limma/', suffix = '_HMOX.csv')

## FLT1/PGF, specific to P. falciparum infection - Figure 3

In [ ]:
genes_input= c(
    #-- FLT1/PGF balance
    "FLT1","PGF"
)

recoverPvalLog2FC(files, columns, genes=genes_input, outdir='../results/tables_limma/', suffix = '_selectedFLT1PGF.csv')


-----

### Other genes tested